<a href="https://colab.research.google.com/github/PRsant90/Challenge_datos_2_parte_2/blob/main/Telecomx_parte_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importar liberias

In [1]:
# Instalar la librería imblearn
%pip install imblearn

# Importar las librerías necesarias
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, RobustScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve
from imblearn.over_sampling import SMOTE
import matplotlib.pyplot as plt
import seaborn as sns

Cargar datos anteriores

In [2]:
df = pd.read_csv('/content/df_final.csv')
df.head()

,id_cliente,evasión,sexo,adulto_mayor,tiene_pareja,tiene_dependientes,antiguedad_meses,servicio_telefono,lineas_multiples,tipo_internet,...,proteccion_dispositivos,soporte_tecnico,usa_streaming_tv,usa_streaming_pelis,tipo_contrato,factura_online,metodo_pago,cargos_mensuales,cargos_totales,cuentas_diarias
0,0002-orfbo,0,0,0,1,1,9,1,no,dsl,...,0,1,1,0,one year,1,mailed check,65.6,593.30,2.186667
1,0003-mknfe,0,1,0,0,0,9,1,yes,dsl,...,0,0,0,1,month-to-month,0,mailed check,59.9,542.40,1.996667
2,0004-tlhlj,1,1,0,0,0,4,1,no,fiber optic,...,1,0,0,0,month-to-month,1,electronic check,73.9,280.85,2.463333
3,0011-igkff,1,1,1,1,0,13,1,no,fiber optic,...,1,0,1,1,month-to-month,1,electronic check,98.0,1237.85,3.266667
4,0013-exchz,1,0,1,1,0,3,1,no,fiber optic,...,0,1,1,0,month-to-month,1,mailed check,83.9,267.40,2.796667
